In [1]:
import numpy as np
import os
import pandas as pd
import PIL
import PIL.Image
from sklearn.utils import shuffle
from matplotlib import pyplot as plt

from keras.models import Sequential, Model
from keras.layers import Input, merge, Activation
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.callbacks import CSVLogger, ModelCheckpoint

import base64
from datetime import datetime
import os
import shutil
import numpy as np
import socketio
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [2]:

#initialize our server
sio = socketio.Server()
#our flask (web) app
app = Flask(__name__)
#init our model and image array as empty
model = None
prev_image_array = None

#set min/max speed for our autonomous car
MAX_SPEED = 25
MIN_SPEED = 10

#and a speed limit
speed_limit = MAX_SPEED

In [9]:
def crop(image):
    return image[60:-25, :, :]

def get_model():
    
    img_size = (75, 320)
    
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(3,) + img_size, output_shape = (3,) + img_size))
    model.add(Convolution2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(64, 3, 3, activation='elu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(Convolution2D(64, 3, 3, activation='elu'))
    model.add(BatchNormalization(axis=1))
    
    model.add(Flatten())
    
    model.add(Dense(100, activation='elu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(50, activation='elu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10, activation='elu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
    
    return model

In [11]:

#registering event handler for the server
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = float(data["steering_angle"])
        # The current throttle of the car, how hard to push peddle
        throttle = float(data["throttle"])
        # The current speed of the car
        speed = float(data["speed"])
        # The current image from the center camera of the car
        image = Image.open(BytesIO(base64.b64decode(data["image"])))
        try:
            image = np.asarray(image)       # from PIL image to numpy array
            image = crop(image) # apply the preprocessing
            image = np.array([image])       # the model expects 4D array
            image = np.transpose(image,(0,3,1,2))
            
            # predict the steering angle for the image
            steering_angle = float(model.predict(image, batch_size=1))
            # lower the throttle as the speed increases
            # if the speed is above the current speed limit, we are on a downhill.
            # make sure we slow down first and then go back to the original max speed.
            global speed_limit
            if speed > speed_limit:
                speed_limit = MIN_SPEED  # slow down
            else:
                speed_limit = MAX_SPEED
            throttle = 1.0 - steering_angle**2 - (speed/speed_limit)**2

            print('{} {} {}'.format(steering_angle, throttle, speed))
            send_control(steering_angle, throttle)
        except Exception as e:
            print(e)
    else:
        
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)

In [12]:
model = get_model()
model.load_weights("./model_temp.h5")

In [ ]:

# wrap Flask application with engineio's middleware
app = socketio.Middleware(sio, app)

# deploy as an eventlet WSGI server
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


(649) wsgi starting up on http://0.0.0.0:4567
(649) accepted ('172.17.0.1', 54278)


('connect ', '58f70529331f424688d67acddd0a415a')
-0.0422349497676 0.998216209018 0.0
-0.0433053709567 0.998124644846 0.0
-0.0433053709567 0.998124644846 0.0
-0.0435689650476 0.998101745285 0.0
-0.0435689650476 0.998101745285 0.0
-0.0435689650476 0.998101745285 0.0
-0.0143524110317 0.989874645082 2.4899
-0.0143524110317 0.989874645082 2.4899
-0.0143524110317 0.989874645082 2.4899
-0.0205612927675 0.96246706364 4.816
-0.0205612927675 0.96246706364 4.816
-0.0205612927675 0.96246706364 4.816
-0.0321690067649 0.92348968102 6.8682
-0.0321690067649 0.92348968102 6.8682
-0.0321690067649 0.92348968102 6.8682
-0.0335486717522 0.855382016208 9.4701
-0.0335486717522 0.855382016208 9.4701
-0.0335486717522 0.855382016208 9.4701
-0.00817907508463 0.785641879675 11.5729
-0.00817907508463 0.785641879675 11.5729
-0.00817907508463 0.785641879675 11.5729
0.0135500300676 0.732965632909 12.9144
0.0135500300676 0.732965632909 12.9144
0.0135500300676 0.732965632909 12.9144
-0.0324674099684 0.67274496769 14.27

-0.0245173703879 0.191262625253 22.4741
-0.0245173703879 0.191262625253 22.4741
-0.0211558267474 0.199436492931 22.3623
-0.0211558267474 0.199436492931 22.3623
-0.0211558267474 0.199436492931 22.3623
-0.00842093862593 0.191584241393 22.477
-0.00842093862593 0.191584241393 22.477
-0.00842093862593 0.191584241393 22.477
-0.000705541810021 0.190503423811 22.493
-0.000705541810021 0.190503423811 22.493
-0.000705541810021 0.190503423811 22.493
-0.0068848580122 0.19824725433 22.3845
-0.0068848580122 0.19824725433 22.3845
-0.0068848580122 0.19824725433 22.3845
-0.0298835001886 0.197387305872 22.3847
-0.0298835001886 0.197387305872 22.3847
-0.0298835001886 0.197387305872 22.3847
-0.00113814231008 0.189321763256 22.5094
-0.00113814231008 0.189321763256 22.5094
-0.00113814231008 0.189321763256 22.5094
-0.0171653982252 0.187983635168 22.5239
-0.0171653982252 0.187983635168 22.5239
-0.0171653982252 0.187983635168 22.5239
0.00169748603366 0.188520446141 22.5205
0.00169748603366 0.188520446141 22.52

-0.0294593051076 0.193338673743 22.4415
-0.0294593051076 0.193338673743 22.4415
-0.0294593051076 0.193338673743 22.4415
-0.00131627358496 0.201370071824 22.3415
-0.00131627358496 0.201370071824 22.3415
-0.00131627358496 0.201370071824 22.3415
-0.0332517214119 0.191958619407 22.4574
-0.0332517214119 0.191958619407 22.4574
-0.0332517214119 0.191958619407 22.4574
-0.0631817057729 0.18697258476 22.4866
-0.0631817057729 0.18697258476 22.4866
-0.0631817057729 0.18697258476 22.4866
-0.0411622934043 0.196614647218 22.3843
-0.0411622934043 0.196614647218 22.3843
-0.0411622934043 0.196614647218 22.3843
-0.183010950685 0.165618026154 22.3731
-0.183010950685 0.165618026154 22.3731
-0.183010950685 0.165618026154 22.3731
-0.165807783604 0.163227618896 22.49
-0.165807783604 0.163227618896 22.49
-0.165807783604 0.163227618896 22.49
-0.187921345234 0.161690358005 22.4025
-0.187921345234 0.161690358005 22.4025
-0.187921345234 0.161690358005 22.4025
-0.128069445491 0.193604335276 22.2204
-0.128069445491 

-0.0467492826283 0.19509168587 22.3987
-0.0467443615198 0.187620553002 22.5027
-0.0467443615198 0.187620553002 22.5027
-0.0407085306942 0.194895907625 22.4088
-0.0171769950539 0.189006400297 22.5097
-0.00860445667058 0.196421687869 22.4096
-0.00860445667058 0.196421687869 22.4096
-0.0411240421236 0.194747168103 22.4104
-0.0204031318426 0.196222851795 22.4076
-0.0366049855947 0.18801194543 22.509
-0.0366049855947 0.18801194543 22.509
-0.0766759514809 0.183371825409 22.5104
-0.0316824316978 0.194910986258 22.4177
-0.0315447859466 0.19504163688 22.416
-0.0315447859466 0.19504163688 22.416
-0.0418018624187 0.187539647482 22.5099
-0.027738314122 0.195783678026 22.4088
-0.0271032266319 0.18806977421 22.5166
-0.0271032266319 0.18806977421 22.5166
-0.0337939038873 0.187748792604 22.5154
-0.0367154255509 0.194789108471 22.4146
-0.059870518744 0.185169442121 22.5173
-0.059870518744 0.185169442121 22.5173
-0.0744808614254 0.183401060225 22.5146
-0.0235581509769 0.195381297107 22.4174
-0.045754835

...

-0.0230678264052 0.187292016121 22.5302
-0.0230678264052 0.187292016121 22.5302
-0.0349340811372 0.193854834999 22.4294
-0.00462724734098 0.195369588582 22.425
-0.00462724734098 0.195369588582 22.425
-0.0134973330423 0.195682368305 22.4184
-0.0317979194224 0.187375289504 22.5224
-0.0317979194224 0.187375289504 22.5224
-0.0614266358316 0.184151841306 22.5288
-0.0858533009887 0.180237047125 22.5332
-0.0858533009887 0.180237047125 22.5332
-0.0964188873768 0.185950871757 22.427
-0.0817604959011 0.189538422014 22.4134
-0.0817604959011 0.189538422014 22.4134
-0.085465721786 0.189936953376 22.3992
-0.0853652432561 0.18259037062 22.5017
-0.0853652432561 0.18259037062 22.5017
-0.0890371203423 0.190051842065 22.3889
-0.0254742354155 0.189401462546 22.4993
-0.0254742354155 0.189401462546 22.4993
-0.0299959667027 0.189640151982 22.4925
-0.014153256081 0.190433161438 22.4912
-0.014153256081 0.190433161438 22.4912
-0.0282644648105 0.189920960029 22.49
-0.0447499938309 0.196965278036 22.3751
-0.04474

-0.0314411707222 0.191745142384 22.462
-0.0314411707222 0.191745142384 22.462
-0.0227267444134 0.200433436272 22.3474
-0.0379357300699 0.19045337368 22.4737
-0.0379357300699 0.19045337368 22.4737
-0.00969229172915 0.199868834777 22.3612
-0.00841594673693 0.191426079617 22.4792
-0.00841594673693 0.191426079617 22.4792
-0.00578330596909 0.191413106956 22.4799
0.0121373375878 0.198770404012 22.3758
0.0121373375878 0.198770404012 22.3758
-0.0107399160042 0.19073403142 22.4882
0.0262706615031 0.198027071608 22.3786
0.0262706615031 0.198027071608 22.3786
-0.00391695834696 0.190560555037 22.492
0.00852757226676 0.198064341487 22.3867
0.00852757226676 0.198064341487 22.3867
0.013164379634 0.190143468133 22.4956
-0.00960692856461 0.19760771998 22.3928
-0.00960692856461 0.19760771998 22.3928
-0.00191491097212 0.197216172716 22.3995
-0.0333363004029 0.188679877619 22.5029
-0.0333363004029 0.188679877619 22.5029
-0.049105040729 0.194550471599 22.4031
-0.0344196073711 0.187987079028 22.5115
-0.0344

-0.0533911511302 0.197434129157 22.3567
-0.0467765815556 0.198704682234 22.3482
-0.0468504577875 0.191171130221 22.4532
-0.0468504577875 0.191171130221 22.4532
-0.0429093800485 0.191323687504 22.456
-0.0914646908641 0.184296019925 22.463
-0.091356664896 0.192403655715 22.3502
-0.091356664896 0.192403655715 22.3502
-0.029095556587 0.199853079291 22.3509
-0.0248138606548 0.192527616815 22.4563
-0.0558468550444 0.198510286526 22.3379
-0.0558468550444 0.198510286526 22.3379
-0.0498110428452 0.198819170011 22.3425
-0.0326888002455 0.191701070739 22.4615
-0.0328746587038 0.199325616815 22.355
-0.0328746587038 0.199325616815 22.355
-0.0194813385606 0.191699941112 22.4711
-0.0582365319133 0.196163361375 22.3669
-0.0336488373578 0.198100495008 22.3714
-0.0336488373578 0.198100495008 22.3714
-0.0798892453313 0.185229705664 22.4776
-0.0421739928424 0.197045987272 22.3771
-0.0472924523056 0.196344575555 22.3805
-0.0472924523056 0.196344575555 22.3805
-0.0351746417582 0.189957503681 22.4834
-0.0338

-0.0117438668385 0.185846717608 22.5557
-0.0117438668385 0.185846717608 22.5557
-0.00776659604162 0.194806746722 22.4323
-0.00614098273218 0.184553288331 22.575
-0.00589696643874 0.184520105387 22.5755
-0.00589696643874 0.184520105387 22.5755
-0.00650530401617 0.196338664316 22.4112
-0.00805107783526 0.184475611362 22.5757
-0.0067091146484 0.184676014597 22.5732
-0.0067091146484 0.184676014597 22.5732
-0.00500808190554 0.198018848716 22.388
0.00452653691173 0.184938892848 22.5699
-0.00133926072158 0.185217573277 22.5663
-0.00133926072158 0.185217573277 22.5663
-0.00408349419013 0.199044242771 22.3738
-0.00484743434936 0.185549670444 22.5614
-0.00530665321276 0.200020479432 22.36
-0.00530665321276 0.200020479432 22.36
-0.00846740510315 0.199755116475 22.3631
-0.0104310289025 0.200454925092 22.3528
-0.00109982280992 0.200304996854 22.3564
-0.00109982280992 0.200304996854 22.3564
-0.00504723517224 0.186024120553 22.5548
-0.0116843702272 0.199038942948 22.3722
-0.0132855176926 0.1987125727

wsgi exiting
